In [1]:
# Import modules
import osmnx as ox 

import numpy as np
import pandas as pd
import geopandas as gpd

from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry import LineString, MultiLineString

In [2]:
# Specify type of data
tags = {'building': True}

# Download building geometries from OSM
gdf = ox.geometries_from_place('Eugene, Oregon, USA', tags)

C:\Users\ajhth\anaconda3\envs\lab3\lib\site-packages\osmnx\geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))
C:\Users\ajhth\anaconda3\envs\lab3\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
C:\Users\ajhth\anaconda3\envs\lab3\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):


In [3]:
# Filter cafes
cafes = gdf[gdf['amenity'] == 'cafe'].reset_index()
cafes

,element_type,osmid,addr:state,building,ele,gnis:county_id,gnis:created,gnis:feature_id,name,operator,...,house:soliciting,soliciting,covered,facebook,opening_hours:url,yelp,surface,ways,type,contact:website
0,way,123376983,NaN,yes,NaN,NaN,NaN,NaN,Latitude 10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,way,123493968,NaN,yes,NaN,NaN,NaN,NaN,The Human Bean,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,way,221500325,NaN,retail,NaN,NaN,NaN,NaN,Brail's Expresso,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,way,244977193,NaN,yes,NaN,NaN,NaN,NaN,TopCity Frozen Yogurt Cafe,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,way,244983219,NaN,yes,NaN,NaN,NaN,NaN,Starbucks,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,way,277456563,NaN,yes,NaN,NaN,NaN,NaN,Espresso Roma,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,way,283863396,NaN,yes,NaN,NaN,NaN,NaN,Dutch Bros. Coffee,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,way,385083705,NaN,commercial,NaN,NaN,NaN,NaN,Greenleaf Juicing Company,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,way,414520035,NaN,yes,NaN,NaN,NaN,NaN,Romano's,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,way,420982688,NaN,yes,NaN,NaN,NaN,NaN,Voodoo Doughnut,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Reproject to UTM Zone 10N
gdf = gdf.to_crs('EPSG:32610')
cafes = cafes.to_crs('EPSG:32610')

In [5]:
# Get coordinates of Condon Hall
condon_hall = gdf[gdf['name'] == 'Condon Hall'].reset_index()

In [6]:
# Get cafe and Condon Hall centroids
cafes['centroid'] = cafes['geometry'].apply(
  lambda x: x.centroid if type(x) == Polygon else (
  x.centroid if type(x) == MultiPolygon else x))

condon_hall['centroid'] = condon_hall['geometry'].apply(
  lambda x: x.centroid if type(x) == Polygon else (
  x.centroid if type(x) == MultiPolygon else x))

In [7]:
# Compute distances
condon_hall_x = condon_hall['centroid'].x.values[0]
condon_hall_y = condon_hall['centroid'].y.values[0]
distances = np.sqrt(((condon_hall_x - cafes['centroid'].x.values)**2)
                     + ((condon_hall_y - cafes['centroid'].y.values)**2))

# Add to GeoDataFrame
cafes['euclidean_distance'] = distances

In [8]:
print(cafes.nsmallest(10, ['euclidean_distance'])[['name', 'euclidean_distance']])

                         name  euclidean_distance
7   Greenleaf Juicing Company           53.254590
5               Espresso Roma          135.286656
10         Dutch Bros. Coffee          830.863412
16            The Rabbit Hole          994.588162
4                   Starbucks         1094.039455
2            Brail's Expresso         1225.004530
9             Voodoo Doughnut         1240.194485
15              Amazon Coffee         1296.503685
33                  Cafe Yumm         1335.694760
34          Magpie Coffeeshop         1349.524658


In [9]:
# Make a new DataFrame containing only the three most relevant columns
nearest_cafes = cafes.nsmallest(10, ['euclidean_distance'])[['name', 'euclidean_distance', 'centroid']]

# Set column geometry
nearest_cafes = nearest_cafes.set_geometry('centroid')

# Convert back to WGS84
nearest_cafes = nearest_cafes.to_crs('EPSG:4326')

# Import package
import folium

# Define center of map (i.e. Condon Hall) and initial zoom level
lat_lon = [44.0451, -123.0781]
m = folium.Map(location=lat_lon, zoom_start=15)

for i in range(0, nearest_cafes.shape[0]):
    my_string = 'name: {}, distance: {}'.format(nearest_cafes.iloc[i]['name'], nearest_cafes.iloc[i]['euclidean_distance'])
    folium.Marker([nearest_cafes.iloc[i]['centroid'].y, nearest_cafes.iloc[i]['centroid'].x],
                 popup=my_string).add_to(m)

# Display map
m